# Toronto COVID-19 Cases Project

Author: Adrienne Siu
Date: August 2020

# 1. Problem Definition

In this project, I will use machine learning to:
(1) Predict the outcomes of cases of COVID-19 in Toronto
(2) Find the variables that correlate most with the outcome

The dataset is available on Toronto Open Data and the version from July 29, 2020 was used: https://open.toronto.ca/dataset/covid-19-cases-in-toronto/

This dataset has been saved as 'COVID19 cases.csv'.

The three possible outcomes are: fatal, resolved (not fatal), and active.

In [14]:
# DELETE
%qtconsole
import pdb
#import contextlib
#with contextlib.redirect_stdout(None):
#import pixiedust

#%%pixie_debugger
#pdb.set_trace()

# 2. Exploratory Data Analysis (EDA)

In this section, I will explore the data, including:
(1) An overall view of the data and types of variables
(2) splitting the data into the training and validation sets
(3) finding correlations between variables and outcomes.

## Overall view of the data and types of variables

In [15]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import math as mt
from datetime import datetime as dt

In [16]:
# Path of dataset file (.csv)
covid_file_path = 'COVID19 cases.csv'

# Read the file
covid_data = pd.read_csv(covid_file_path)

In [17]:
# Display the first few rows of the dataset
covid_data.head()

,_id,Assigned_ID,Outbreak Associated,Age Group,Neighbourhood Name,FSA,Source of Infection,Classification,Episode Date,Reported Date,Client Gender,Outcome,Currently Hospitalized,Currently in ICU,Currently Intubated,Ever Hospitalized,Ever in ICU,Ever Intubated
0,1,1,Sporadic,50 to 59 Years,Willowdale East,M2N,Travel,CONFIRMED,2020-01-22,2020-01-23,FEMALE,RESOLVED,No,No,No,No,No,No
1,2,2,Sporadic,50 to 59 Years,Willowdale East,M2N,Travel,CONFIRMED,2020-01-21,2020-01-23,MALE,RESOLVED,No,No,No,Yes,No,No
2,3,3,Sporadic,20 to 29 Years,Parkwoods-Donalda,M3A,Travel,CONFIRMED,2020-02-05,2020-02-21,FEMALE,RESOLVED,No,No,No,No,No,No
3,4,4,Sporadic,60 to 69 Years,Church-Yonge Corridor,M4W,Travel,CONFIRMED,2020-02-16,2020-02-25,FEMALE,RESOLVED,No,No,No,No,No,No
4,5,5,Sporadic,60 to 69 Years,Church-Yonge Corridor,M4W,Travel,CONFIRMED,2020-02-20,2020-02-26,MALE,RESOLVED,No,No,No,No,No,No


In [18]:
# Look for categorical variables
covid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15338 entries, 0 to 15337
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   _id                     15338 non-null  int64 
 1   Assigned_ID             15338 non-null  int64 
 2   Outbreak Associated     15338 non-null  object
 3   Age Group               15311 non-null  object
 4   Neighbourhood Name      14729 non-null  object
 5   FSA                     14775 non-null  object
 6   Source of Infection     15338 non-null  object
 7   Classification          15338 non-null  object
 8   Episode Date            15338 non-null  object
 9   Reported Date           15338 non-null  object
 10  Client Gender           15338 non-null  object
 11  Outcome                 15338 non-null  object
 12  Currently Hospitalized  15338 non-null  object
 13  Currently in ICU        15338 non-null  object
 14  Currently Intubated     15338 non-null  object
 15  Ev

Apart from the patient ID, the columns are all categorical variables. I will deal with encoding for categorical variables in the modelling section.

In [19]:
# Change Episode Date and Reported Date columns to datetimes
# Input: data: dataframe
# Output: dataframe with date columns as datetime objects
def change_to_datetime(data):
    data['Episode Date'] = pd.to_datetime(data['Episode Date'], format='%Y-%m-%d')
    data['Reported Date'] = pd.to_datetime(data['Reported Date'], format='%Y-%m-%d')
    return data

In [20]:
# Change Episode Date and Reported Date columns to datetimes
covid_data = change_to_datetime(covid_data)

#covid_data['Episode Date'].dt.strftime('%m')

In [21]:
# Check for duplicated entries
covid_data.duplicated().sum()

0

## Define training and validation sets

In [22]:
from sklearn.model_selection import train_test_split

# Target: outcome
y = covid_data.Outcome

# Predictor: drop _id, Assigned_ID, and Outcome columns
X = covid_data.drop(['_id', 'Assigned_ID', 'Outcome'], axis=1)

# Split into training and validation sets (80/20 split)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [9]:
# X_train.to_excel("X_train.xlsx")
# X_valid.to_excel("X_valid.xlsx")

## Describe variables in the training set

In [27]:
X_train.describe()

,Outbreak Associated,Age Group,Neighbourhood Name,FSA,Source of Infection,Classification,Episode Date,Reported Date,Client Gender,Currently Hospitalized,Currently in ICU,Currently Intubated,Ever Hospitalized,Ever in ICU,Ever Intubated
count,12270,12247,11787,11824,12270,12270,12270,12270,12270,12270,12270,12270,12270,12270,12270
unique,2,9,140,96,8,2,161,154,5,2,2,2,2,2,2
top,Sporadic,50 to 59 Years,Glenfield-Jane Heights,M9V,N/A - Outbreak associated,CONFIRMED,2020-04-15 00:00:00,2020-05-29 00:00:00,FEMALE,No,No,No,No,No,No
freq,7681,1958,400,705,4589,11297,246,357,6490,12200,12255,12262,10757,11939,12036
first,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-21 00:00:00,2020-01-23 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-27 00:00:00,2020-07-27 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
y_train.describe()

count        12270
unique           3
top       RESOLVED
freq         11018
Name: Outcome, dtype: object

In [10]:
# Function to obtain filtered entries in dataset
# Input: column_name is a string for the name of the column, e.g. 'Outbreak', 'Age Group'
#        filter_column is a string for the column to be filtered
#        filter_entry is a string for the entry to be filtered
# Ex. To filter only the fatal outcomes, filter_column = 'Outcome' and filter_entry = 'FATAL'
# Returns entries and value counts for the specific column after filtering
def get_filtered_entries_and_value_counts_from_column(column_name, filter_column, filter_entry):
    # Get the count of each unique entry (ordered by name)
    entry_vc = X_train[X_train[filter_column]==filter_entry][column_name].value_counts() #.sort_index()
    # Get the unique entries and put them in a list
    entry = entry_vc.index.tolist()    
    return entry, entry_vc

In [11]:
# Function to create a bar chart of a column variable with filters applied
# Stack by outcome: active, fatal, resolved
# Input: column_name is a string for the name of the column, e.g. 'Outbreak', 'Age Group'
#        filter_column is a string for the column to be filtered
#        filter_entry_list is a list of strings for entries to be filtered
def create_filtered_bar_chart(column_name, filter_column, filter_entry_list, fig_size):
    # Initialize lists of entries and value counts after filtering
    entry_list = []
    entry_vc_list = []
    
    # Figure size
    if fig_size == 'large':
        fig = plt.figure(figsize=(20,30))
        #plt.xticks(rotation=90)
    elif fig_size == 'medium':
        fig = plt.figure(figsize=(15,15))
    else:
        fig = plt.figure()

    # Loop through entries to be filtered
    for i in range(len(filter_entry_list)):
        # Entries and value counts for each filter
        entry, entry_vc = get_filtered_entries_and_value_counts_from_column(column_name, filter_column, filter_entry_list[i])
        # Append to list
        entry_list.append(entry)
        entry_vc_list.append(entry_vc)
        # Create a bar stacked for each filtered entry, e.g. 'Fatal', 'Active', 'Resolved'
        # Make vertical bar charts for Episode Date and Reported Date columns
        if column_name == 'Episode Date' or column_name == 'Reported Date':
            plt.bar(entry_list[i], entry_vc_list[i])
            plt.xticks(rotation=45)
        # Otherwise make horizontal bar charts
        else:
            plt.barh(entry_list[i], entry_vc_list[i])
            # Invert the y-axis so that the order of the entries is from top to bottom
            plt.gca().invert_yaxis()
        
    plt.title('Number of COVID-19 Cases By ' + column_name)
    plt.legend(filter_entry_list)
    plt.savefig('Plots/' + column_name + ' ' + str(i) + '.pdf')
    plt.show()

In [12]:
# Put the column names of the dataset into a list
columns = X_train.columns

# Remove the first two column names (the IDs) because I don't need to graph them
columns_no_IDs = columns[2:]

# Column to be filtered
filter_column = 'Outcome'
# Entries to be filtered:
# First make stacked bar charts of all three outcomes, then bar charts of only fatal outcomes
filter_entry_list = [['FATAL', 'RESOLVED', 'ACTIVE'], ['FATAL']]

for j in range(len(filter_entry_list)):
    # Create bar charts for relevant columns
    for column in columns_no_IDs:
        #create_bar_chart(column)
        # Figure size
        if column == 'Neighbourhood Name' or column == 'FSA': # or column == 'Episode Date' or column == 'Reported Date':
            fig_size = 'large'
        elif column == 'Episode Date' or column == 'Reported Date':
            fig_size = 'medium'
        else:
            fig_size = 'normal'

        create_filtered_bar_chart(column, filter_column, filter_entry_list[j], fig_size)

KeyError: 'Outcome'

<Figure size 1440x2160 with 0 Axes>

The fatal cases make up 7.5% of total cases. As the number of resolved cases is an order of a magnitude higher, it is difficult to see the fatal cases in the bar charts. Hence, the fatal cases are graphed separately.

# 3. Data Cleaning

In this section, I will clean the data for analysis.

In [ ]:
# Function to create a table of missing values in columns
# Function written by Nikos Tavoularis on Stack Overflow: https://stackoverflow.com/questions/26266362/how-to-count-the-nan-values-in-a-column-in-pandas-dataframe/39734251#39734251
# Function comments by AS
def missing_values_table(df):
        # Count the number of missing values in each column
        mis_val = df.isnull().sum()
        # Calculate the percentage of missing values in each column
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        # Put the number of missing values and % missing values in a table
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        # Rename the columns of the table
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        # Take out columns with no missing values
        # Reorder columns with missing values in descending order
        # Round % missing values to 1 decimal place
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        # Print summary
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        # Return table of missing values
        return mis_val_table_ren_columns

In [ ]:
# Create table of missing values
missing_values_table(covid_data)

As observed, there are only 3/18 columns with missing values. For each column, the % missing values is 4% or less. Hence, the dataset is fairly complete.

I will fill in the missing values as follows:

-- If both the neighbourhood name and FSA are blank, find the modal FSA and neighbourhood name (as a pair).
Note I am not using SimpleImputer because I would like to consider the relationship between the neighbourhood name and FSA. If the modal neighbourhood name and modal FSA are treated separately as in SimpleImputer, the resulting pair
of neighbourhood name and FSA could be impossible in real life.

-- If the neighbourhood name is blank but the FSA is known, find the modal neighbourhood name for the given FSA.

-- Note that there are no cases of the FSA being blank but the neighbourhood name is known.

-- The age group will be filled with modal imputation, i.e. the modal age group. Note that the median and the modal age group in the training set are the same, so median imputation would also yield the same result.

In [ ]:
# Functions for neighbourhood name and FSA imputation
# If both the neighbourhood and FSA are blank, find the modal FSA and neighbourhood name (as a pair)
# Input: dataset for imputation
# Output: modal FSA and neighbourhood name, sorted dataset
def find_mode_fsa_nn(data):
    # Group data by FSA and neighbourhood and count number of occurrences
    fsa_nn_group_data = data.groupby(['FSA', 'Neighbourhood Name'])['Outbreak Associated'].agg([len]).reset_index()
    # Sort from highest to lowest count (i.e. the mode is the first value)
    fsa_nn_group_data_sorted = fsa_nn_group_data.sort_values(by='len', ascending=False)
    # Find modal neighbourhood name and FSA
    mode_fsa = fsa_nn_group_data_sorted.iloc[0]['FSA']
    mode_nn = fsa_nn_group_data_sorted.iloc[0]['Neighbourhood Name']
    return mode_fsa, mode_nn, fsa_nn_group_data_sorted

# Impute missing values for the neighbourhood name and FSA
# Note there are no cases when the neighbourhood name is known but the FSA is blank
# Input: dataset, sorted dataset
def impute_missing_fsa_nn(data, mode_fsa, mode_nn, fsa_nn_group_data_sorted):
    # If an entry is blank, it is a 'NaN' and hence a float
    # If an entry is not blank, it is a string
    for i in data.index:
         # If both the neighbourhood name and FSA are blank
        if type(data['Neighbourhood Name'][i]) == float and type(data['FSA'][i]) == float:
            # Use the modal FSA and neighbourhood name
            data['FSA'][i] = mode_fsa
            data['Neighbourhood Name'][i] = mode_nn
            #print("Instance 1", i, data['FSA'][i], data['Neighbourhood Name'][i])

         # If the neighbourhood name is blank but the FSA is known
        if type(data['Neighbourhood Name'][i]) == float and type(data['FSA'][i]) != float:
            # Find the modal neighbourhood name for the given FSA
            # i.e. find the first instance of FSA in fsa_nn_group_data_sorted and its associated neighbourhood name
            mode_nn_for_fsa = fsa_nn_group_data_sorted.loc[fsa_nn_group_data_sorted['FSA'] == data['FSA'][i]].iloc[0]
            data['Neighbourhood Name'][i] = mode_nn_for_fsa['Neighbourhood Name']
            #print("Instance 2", i, data['Neighbourhood Name'][i])
    return data

In [ ]:
# Make new copies of datasets
# y doesn't have any missing values, so imputation is not needed
X_train_imp_area = X_train.copy()
X_valid_imp_area = X_valid.copy()

# On the training set, return the mode FSA, mode neighbourhood name, and sorted training set
mode_fsa, mode_nn, fsa_nn_group_data_sorted = find_mode_fsa_nn(X_train_imp_area)

# Impute missing FSA and neighbourhood name values in the training set
X_train_imp_area = impute_missing_fsa_nn(X_train_imp_area, mode_fsa, mode_nn, fsa_nn_group_data_sorted)

# Impute missing FSA and neighbourhood name values in the validation set based on the modes
# in the training set
X_valid_imp_area = impute_missing_fsa_nn(X_valid_imp_area, mode_fsa, mode_nn, fsa_nn_group_data_sorted)

In [ ]:
# Age group: modal imputation
from sklearn.impute import SimpleImputer

# Convert all columns to strings to avoid error with data type
X_train_new = X_train_imp_area.applymap(str) # Convert blanks to 'nan'
X_valid_new = X_valid_imp_area.applymap(str) 

# Imputer with most_frequent (modal) strategy (note this removes the original indices of the entries)
imputer = SimpleImputer(missing_values='nan', strategy='most_frequent', verbose=1)
X_train_imp = pd.DataFrame(imputer.fit_transform(X_train_new)) 
X_valid_imp = pd.DataFrame(imputer.transform(X_valid_new))

# Rename columns
X_train_imp.columns = X_train_imp_area.columns
X_valid_imp.columns = X_valid_imp_area.columns

# Put back the original indices
X_train_imp.index = X_train_imp_area.index
X_valid_imp.index = X_valid_imp_area.index

In [ ]:
# fsa_nn_group_data_sorted.to_excel("fsa_nn_group_data_sorted.xlsx")
# X_train_imp_area.to_excel("X_train_imp_area.xlsx")
# X_valid_imp_area.to_excel("X_valid_imp_area.xlsx")
# X_train_imp.to_excel("X_train_imp.xlsx")
# X_valid_imp.to_excel("X_valid_imp.xlsx")
# X_train_new.to_excel("X_train_new.xlsx")
# X_valid_new.to_excel("X_valid_new.xlsx")

In [ ]:
# Check for missing values
missing_values_table(X_train_imp)
missing_values_table(X_valid_imp)

Missing values have been removed.

# 4. Feature Selection and Engineering

Feature engineering will involve encoding the features:

-- Outbreak Associated (2 values): one-hot encoding with binary values, equivalent to label encoding (0/1) to avoid the dummy variable trap

-- Age Group (9 values): label encoding

-- Neighbourhood Name and FSA: one-hot encoding with 'Other' column for rare cases

-- Source of Infection (8 values): one-hot encoding

-- Classification (2 values): one-hot encoding with binary values, equivalent to label encoding (0/1)

-- Episode Date and Reported Date: encode month, date, day of the week as cyclical features (other possibilities: time since interesting event? holiday? season?)

-- Client Gender (5 values): one-hot encoding

-- Currently Hospitalized, Currently in ICU, Currently Intubated, Ever Hospitalized, Ever in ICU, Ever Intubated (2 values): one-hot encoding with binary values, equivalent to label encoding (0/1)

CONSIDER SCALING AT THE END (FOR THE AGE GROUP)

Feature selection will involve removing unnecessary features.

In [ ]:
# Outbreak Associated, Age Group, Currently Hospitalized, Currently in ICU, Currently Intubated, 
# Ever Hospitalized, Ever in ICU, Ever Intubated: label encoding

from sklearn.preprocessing import LabelEncoder

# Specify columns for label encoding
lbl_enc_col = ['Outbreak Associated', 'Age Group', 'Classification', 'Currently Hospitalized', 'Currently in ICU', \
               'Currently Intubated', 'Ever Hospitalized', 'Ever in ICU', 'Ever Intubated']

# Define the label encoder
lbl_enc = LabelEncoder()

# Make a copy
X_train_lbl = X_train_imp.copy()
X_valid_lbl = X_valid_imp.copy()

# Loop through all of the specified columns
for col in lbl_enc_col:
    X_train_lbl[col] = lbl_enc.fit_transform(X_train_lbl[col])
    X_valid_lbl[col] = lbl_enc.transform(X_valid_lbl[col])
    
# Put back indices
X_train_lbl.index = X_train_imp.index
X_valid_lbl.index = X_valid_imp.index

In [ ]:
# # Neighbourhood Name and FSA: one-hot encoding with 'Other' column for rare cases
# # Based on Maya Gilad's tutorial here: https://medium.com/gett-engineering/handling-rare-categorical-values-in-pandas-d1e3f17475f0

# # Value counts (number of cases associated with each FSA)
# X_train_oh['FSA'].value_counts()
# # Statistics
# X_train_oh['FSA'].value_counts().describe()
# # Normalized value counts
# X_train_oh['FSA'].value_counts(normalize=True)
# # Normalized value counts: statistics
# X_train_oh['FSA'].value_counts(normalize=True).describe()

In [ ]:
# # Value counts (number of cases associated with each Neighbourhood Name)
# X_train_oh['Neighbourhood Name'].value_counts()
# # Statistics
# X_train_oh['Neighbourhood Name'].value_counts().describe()
# # Normalized value counts
# X_train_oh['Neighbourhood Name'].value_counts(normalize=True)
# # Normalized value counts: statistics
# X_train_oh['Neighbourhood Name'].value_counts(normalize=True).describe()

In [ ]:
# Function to get the threshold for specifying rare cases and the normalized value counts (in descending order)
# Input: col_name: column name
# Note that this is done on the training set only (X_train_oh)
def get_threshold_and_norm_vc_desc(col_name):    
    # Normalized value counts
    norm_vc = X_train_oh[col_name].value_counts(normalize=True, ascending=True)

    # Inferred threshold
    # Having 10% of outcomes in the 'Other' category seems to be a good starting point, so I will use this
    # to set the threshold
    threshold = norm_vc[(norm_vc.cumsum() > 0.1).idxmax()]

    # Normalized value counts (in descending order)
    norm_vc_desc = X_train_oh[col_name].value_counts(normalize=True)
    
    return threshold, norm_vc_desc

# Function to replace the column with rare cases marked as 'Other'
# Input: data: either the training set (X_train_oh) or the validation set (X_valid_oh)
#        thresh_fsa, thresh_nn: threshold for marking rare case (from the training set)
#        norm_vc_desc_fsa, norm_vc_desc_nn: normalized value counts in descending order (from the training set)
# Output: data with modified column
def replace_rare_cases_in_col(data, thresh_fsa, thresh_nn, norm_vc_desc_fsa, norm_vc_desc_nn):
    # FSA
    # Map the column to its normalized value count
    map_to_norm_vc_desc_fsa = data['FSA'].map(norm_vc_desc_fsa)
    # Replace the column entry with 'Other' when the normalized value count is less than the threshold
    data['FSA'] = data['FSA'].mask(map_to_norm_vc_desc_fsa < thresh_fsa, 'Other')

    # Neighbourhood Name
    # Map the column to its normalized value count
    map_to_norm_vc_desc_nn = data['Neighbourhood Name'].map(norm_vc_desc_nn)
    # Replace the column entry with 'Other' when the normalized value count is less than the threshold
    data['Neighbourhood Name'] = data['Neighbourhood Name'].mask(map_to_norm_vc_desc_nn < thresh_nn, 'Other')

    return data

In [ ]:
# # Neighbourhood Name and FSA: one-hot encoding with 'Other' column for rare cases
# # Based on Maya Gilad's tutorial here: https://medium.com/gett-engineering/handling-rare-categorical-values-in-pandas-d1e3f17475f0

# FSA and neighbourhood name are treated separately

# Make copies of the dataframes
X_train_oh = X_train_lbl.copy()
X_valid_oh = X_valid_lbl.copy()

# Get thresholds and normalized value counts from the training set
thresh_fsa, norm_vc_desc_fsa = get_threshold_and_norm_vc_desc('FSA')
thresh_nn, norm_vc_desc_nn = get_threshold_and_norm_vc_desc('Neighbourhood Name')

# Get training set and validation set with rare column entries listed as 'Other'
X_train_oh = replace_rare_cases_in_col(X_train_oh, thresh_fsa, thresh_nn, norm_vc_desc_fsa, norm_vc_desc_nn)
X_valid_oh = replace_rare_cases_in_col(X_valid_oh, thresh_fsa, thresh_nn, norm_vc_desc_fsa, norm_vc_desc_nn)


In [ ]:
# # Determine rare cases of the FSA and neighbourhood name
# # A patient's location is defined by a combination of FSA and neighbourhood name.
# # FSA and neighbourhood name are treated as a set, i.e. the number of occurrences of a particular combination of
# # FSA and neighbourhood name are found. 
# # Otherwise, if the FSA and NN are treated independently, you may have cases where one of the FSA and NN are 
# # labelled 'Other', but not both.
# # The threshold for determining 'rare' cases is such that at least 10% of cases are rare. 

# # Make a copy of the dataset
# X_train_other = X_train_oh.copy()
# X_valid_other = X_valid_oh.copy()

# # For the training set: group data by FSA and neighbourhood and count number of occurrences
# fsa_nn = X_train_other.groupby(['Neighbourhood Name', 'FSA'])['Outbreak Associated'].agg([len]).reset_index()
# # Sort from lowest to highest count
# fsa_nn_sorted = fsa_nn.sort_values(by='len', ascending=True)
# # Normalize the value counts (or length)
# norm_vc = fsa_nn_sorted['len']/(np.sum(fsa_nn_sorted['len']))
# ## Map the normalized value counts to the sorted dataframe
# #fsa_nn_map = fsa_nn_sorted['FSA'].map(norm_vc)
# # Add a new column with normalized value counts
# fsa_nn_sorted['norm_vc'] = norm_vc

# # Inferred threshold
# # Having 10% of outcomes in the 'Other' category seems to be a good starting point, so I will use this
# # to set the threshold
# thresh_fsa_nn = norm_vc.loc[(norm_vc.cumsum() > 0.1).idxmax()]

# # Normalized value counts (in descending order)
# fsa_nn_sorted_desc = fsa_nn_sorted.sort_values(by='norm_vc', ascending=False)

In [ ]:
# # Function to return dataframe with rare FSA and Neighbourhood Name cases as 'Other'
# # Output: modified dataframe (either training set or validation set)
# # Input: data: original dataframe (either training set or validation set)
# #        fsa_nn_sorted_desc: dataframe with sorted FSA and NN value counts
# #        thresh_fsa_nn: normalized-value-count threshold for 'Other' cases
# def data_with_fsa_nn_other(data, fsa_nn_sorted_desc, thresh_fsa_nn):
#     # Merge the training data with the normalized value counts
#     norm_vc_merge = data[['Neighbourhood Name', 'FSA']].merge(fsa_nn_sorted_desc, how='left', indicator=True)['norm_vc']
#     # Make the indices the same (norm_vc is ordered by index)
#     norm_vc_merge.index = data.index
#     # Add the norm_vc column to the training data
#     data['norm_vc'] = norm_vc_merge
#     # Replace the Neighbourhood Name and FSA with 'Other' when the normalized value count is less than the threshold
#     data[['Neighbourhood Name', 'FSA']] = data[['Neighbourhood Name', 'FSA']].mask(data['norm_vc'] < thresh_fsa_nn, 'Other')
#     # Remove norm_vc column
#     data = data.drop(['norm_vc'], axis=1)
#     return data

In [ ]:
# # Dataframes with rare FSA and Neighbourhood Name cases as 'Other'
# # Use normalized value counts and threshold for the FSA and NN from the training set and apply to the validation set
# X_train_other = data_with_fsa_nn_other(X_train_other, fsa_nn_sorted_desc, thresh_fsa_nn)
# X_valid_other = data_with_fsa_nn_other(X_valid_other, fsa_nn_sorted_desc, thresh_fsa_nn)

In [ ]:
# Neighbourhood Name, FSA, Source of Infection, Client Gender: one-hot encoding
# FOR TREE-BASED METHODS, VARIABLES SHOULDN'T BE DROPPED...

from sklearn.preprocessing import OneHotEncoder

# Specify columns for one-hot encoding
oh_col = ['Neighbourhood Name', 'FSA', 'Source of Infection', 'Client Gender']

# One-hot encoder:
# Return an array instead of a sparse matrix
# Drop last value to avoid the dummy variable trap and also because the unknown columns aren't very helpful
oh_enc = OneHotEncoder(handle_unknown='error', sparse=False, \
                       drop=[['Other'], ['Other'], ['Unknown/Missing'], ['UNKNOWN']])
X_train_oh_col = pd.DataFrame(oh_enc.fit_transform(X_train_oh[oh_col]))
X_valid_oh_col = pd.DataFrame(oh_enc.transform(X_valid_oh[oh_col]))

# Put back the indices
X_train_oh_col.index = X_train_lbl.index
X_valid_oh_col.index = X_valid_lbl.index


In [ ]:
# Function to rename one-hot encoded columns, dropping last value
# Output: return dataframe with renamed columns
# Input: X_train_oh_col or X_valid_oh_col (one-hot encoded columns);
#        oh_enc (one-hot encoder)
def rename_col_oh(X_oh_col, oh_enc):
    # Concatenate column names, omitting 'Other' or 'Unknown' column names
    X_oh_col.columns = np.concatenate((oh_enc.categories_[0][:-30],  # Neighbourhood Name (before 'Other' at index -30)
                                       oh_enc.categories_[0][-29:],  # Neighbourhood Name (after 'Other')
                                       oh_enc.categories_[1][:-1],   # FSA
                                       oh_enc.categories_[2][:-1],   # Source of Infection
                                       oh_enc.categories_[3][:-1]))  # Client Gender
    return X_oh_col

# Function to put the one-hot encoded columns back into the dataframe
# Keep the original order for now
# Output: return ordered dataframe
# Input: X_train_oh or X_valid_oh (dataframe to modify); X_train_oh_col or X_valid_oh_col (one-hot encoded columns);
#        oh_enc (one-hot encoder)
def order_col_oh(X_oh, X_oh_col, oh_enc):
    X_oh = pd.concat([X_oh.loc[:,'Outbreak Associated':'Age Group'], 
    X_oh_col.loc[:,oh_enc.categories_[0][0]:oh_enc.categories_[0][-31]],   # Neighbourhood Name (before 'Other')
    X_oh_col.loc[:,oh_enc.categories_[0][-29]:oh_enc.categories_[0][-1]],  # Neighbourhood Name (after 'Other')
    X_oh_col.loc[:,oh_enc.categories_[1][0]:oh_enc.categories_[1][-2]],    # FSA
    X_oh_col.loc[:,oh_enc.categories_[2][0]:oh_enc.categories_[2][-2]],    # Source of Infection
    X_oh.loc[:,'Classification':'Reported Date'], 
    X_oh_col.loc[:,oh_enc.categories_[3][0]:oh_enc.categories_[3][-2]],    # Client Gender
    X_oh.loc[:,'Currently Hospitalized':'Ever Intubated']], axis=1, sort=False)
    return X_oh

In [ ]:
# Rename one-hot encoded columns, dropping last value
X_train_oh_col = rename_col_oh(X_train_oh_col, oh_enc)
X_valid_oh_col = rename_col_oh(X_valid_oh_col, oh_enc)

# Put the one-hot encoded columns back into the dataframe
X_train_oh = order_col_oh(X_train_oh, X_train_oh_col, oh_enc)
X_valid_oh = order_col_oh(X_valid_oh, X_valid_oh_col, oh_enc)

In [ ]:
# Episode Date and Reported Date: encoding for dates

# Change Episode Date and Reported Date columns to datetimes
X_train_tmp = change_to_datetime(X_train_oh)
X_valid_tmp = change_to_datetime(X_valid_oh)

In [ ]:
# # Function to extract the month from a dataframe column
# # Input: data: either training or validation set dataframe
# # Output: returns the array with extracted months
# def extract_month(data):
#     # Extract in the form of an integer
#     data['Episode Date'] = data['Episode Date'].dt.strftime('%m').astype(int)
#     data['Reported Date'] = data['Reported Date'].dt.strftime('%m').astype(int)

#     # Rename columns
#     data = data.rename(columns = {'Episode Date':'Episode Month', 'Reported Date':'Reported Month'})
    
#     return data

In [ ]:
# # Extract month
# X_train_month = extract_month(X_train_date)
# X_valid_month = extract_month(X_valid_date)

In [ ]:
# Create a dataframe with just the date info
X_train_ep = X_train_tmp.loc[:,'Episode Date':'Reported Date']

# TO DO: 
# INVESTIGATE IF REPORTED DATE IS A USEFUL FEATURE. IF SO, DO THE SAME FOR THE REPORTED DATE
# DO THE SAME FOR THE VALIDATION SET
# DETERMINE IF OTHER DATE FEATURES ARE NEEDED, E.G. TIME SINCE LOCKDOWN/PHASE 1,2,3

# Extract month
X_train_ep['Episode Month'] = X_train_ep['Episode Date'].dt.strftime('%m').astype(int)

# Make month cyclical
X_train_ep['Episode Month Sin'] = np.sin((X_train_ep['Episode Month']-1)*(2*np.pi/12))
X_train_ep['Episode Month Cos'] = np.cos((X_train_ep['Episode Month']-1)*(2*np.pi/12))

In [ ]:
# Extract day of the month
X_train_ep['Episode Day'] = X_train_ep['Episode Date'].dt.strftime('%d').astype(int)

# Make day of the month cyclical
# 31 days is the max number of days in a month (don't account for differences in months)
X_train_ep['Episode Day Sin'] = np.sin((X_train_ep['Episode Day']-1)*(2*np.pi/31))
X_train_ep['Episode Day Cos'] = np.cos((X_train_ep['Episode Day']-1)*(2*np.pi/31))

In [ ]:
# Extract day of the week
# Sunday is Day 0, Saturday is Day 6
X_train_ep['Episode Weekday'] = X_train_ep['Episode Date'].dt.strftime('%w').astype(int)

# Make day of the month cyclical
# Days numbered from 0, so don't need to subtract 1
# Use 7 in the denominator because there are 7 days in a week
X_train_ep['Episode Weekday Sin'] = np.sin((X_train_ep['Episode Weekday'])*(2*np.pi/7))
X_train_ep['Episode Weekday Cos'] = np.cos((X_train_ep['Episode Weekday'])*(2*np.pi/7))